In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
# set up Chinese fonts
plt.rcParams['font.sans-serif'] = ['SimHei'] # Or any other Chinese characters
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
def to_int(value):
    if value:
        return int(value, 10)
    else:
        return 0

# Convert dictionary for converting digit strings to integers.
convert = dict(zip(['start', 'state', 'lord', 'span', 'notes', 'conqueror'], [to_int, None, None, to_int, None, None]))

# Read table and convert digit strings to int.
df = pd.read_csv("../../db/csv/wangbiao.csv", header=0, converters=convert)

In [ ]:
df['end'] = df['start']+df['span']-1
state_names = df.state.unique()

lb = df.start.min()
ub = df.end.max()
num_rows = ub-lb+1
chronicle_df_data = dict(zip(state_names, ['']*num_rows))
chronicle_df = pd.DataFrame(chronicle_df_data, index=range(lb, ub+1), columns=state_names)

def fill_chronicle(r, cdf):
    ary = []
    for i in range(1, r.end-r.start+2):
        ary.append(f'{r.state},{r.lord},{i}')
    cdf.loc[r.start:r.end, r.state] = ary

df.apply(fill_chronicle, axis=1, cdf=chronicle_df)

In [ ]:
dft = pd.read_csv('../../db/csv/先秦諸侯編年表.csv', index_col=0)
dft.fillna(value='', inplace=True)

In [ ]:
import re
# Reverse dft wuch that 
dftr = dft.iloc[-1::-1]
end_df_list = []
cdict = {}

for c in dftr.columns:
    end_dict = {}

    dc = dftr[c]
    #print(c, dc[dc!=''].index[0])
    end = dc[dc!=''].iloc[0]
    endre = re.findall(r'^(.+),(.+),\d+', end)
    #print(endre)
    end_dict['state'] = endre[0][0]
    end_dict['lord'] = endre[0][1]
    #print(df[df['state']==end_dict['state']])
    #print(df[df['lord']==end_dict['lord']])
    end_df_list.append((df[(df['state']==end_dict['state']) & (df['lord']==end_dict['lord'])]))
    
end_df = pd.concat(end_df_list)

In [ ]:
end_df['s'] = end_df['conqueror'].str.extract(r'.*(.{1})$')
state_order = end_df.sort_values(by='s')

In [ ]:
t = pd.CategoricalDtype(categories=state_order, ordered=True)
df['state1'] = df['state'].astype(t)
df1 = df.sort_values('state1')

In [ ]:
# Function that plots barh chart base on df_x
def plot_barh(df_x, lb, rb, title):
    # Create a figure and axis
    fig, ax = plt.subplots()

    # Set fig size
    fig.set_figwidth(35)
    fig.set_figheight(25)
    plt.rc('font', size=18)          # controls default text sizes

    # Plot colored bars for each row
    for index, row in df_x.iterrows():
        birth_year = row['start']
        lifespan = df_x.loc[index, 'span']
        
        if title == 'state':
            y = row['lord']
        else:
            y = row['state']
        ax.barh(y, lifespan, left=birth_year, color='skyblue')
        state = row['state']
        lord = row['lord']
        ax.text(birth_year + lifespan / 2, y, f'{state} {lord} ({lifespan})', ha='center', va='center', color='black')

    # Set x-axis range from lb to rb
    ax.set_xlim(lb, rb)

    ax.set_xlabel('start')
    
    if title == 'state':
        ax.set_title(state)
    else:
        ax.set_title(title)

    # Show the plot
    plt.show()

In [ ]:
def plot_range(df, lb, ub):
    dfa = df[df.end<ub]
    dfb = dfa[dfa.end>lb]
    dfb = dfb[::-1]
    plot_barh(dfb, lb-20, ub+10, f'From {lb} to {ub}')

In [ ]:
plot_range(df1, -480, -207)

In [ ]:
import re
# Reverse dft wuch that 
dftr = dft.iloc[-1::-1]
end_df_list = []
cdict = {}

for c in dftr.columns:
    end_dict = {}

    dc = dftr[c]
    #print(c, dc[dc!=''].index[0])
    end = dc[dc!=''].iloc[0]
    endre = re.findall(r'^(.+),(.+),\d+', end)
    #print(endre)
    end_dict['state'] = endre[0][0]
    end_dict['lord'] = endre[0][1]
    #print(df[df['state']==end_dict['state']])
    #print(df[df['lord']==end_dict['lord']])
    end_df_list.append((df[(df['state']==end_dict['state']) & (df['lord']==end_dict['lord'])]))
    
end_df = pd.concat(end_df_list)

In [ ]:
import matplotlib.colors as mcolors
tab_colors = list(mcolors.TABLEAU_COLORS.keys())
tab_colors

In [ ]:
def state_end_barh(end_df):
    #print(cser)
    fig, ax = plt.subplots(figsize=(45, 30))
    
    for c, (idx, row) in enumerate(end_df.iterrows()):
        p = ax.barh(row.state, row['end']+480, left=-480, color=tab_colors[c%len(tab_colors)])
        ax.bar_label(p, labels=[f"{row['lord']}"], label_type='center')
        ax.bar_label(p, labels=[f"{row['end']} {row['conqueror']}"], label_type='edge', padding=6)
        
    ax.set_xlabel('公元年')
    ax.set_ylabel('被滅國名')
    
    y2 = ax.secondary_yaxis('right')
    ylabels_2 = [c[0] for c in end_df['conqueror'].str.extract(r'.*(.{1})$').values]
    y2.set_yticks(ax.get_yticks(), labels=ylabels_2)
    y2.set_ylabel('滅國名')

    x2 = ax.secondary_xaxis('top')
    x2.set_xticks([-403, -221], labels=['-403', '-221'])
    x2.set_xlabel('')

    ax.vlines([-403, -221], [0, 0], [18, 18], colors='black', linestyles='solid', label='', data=None)
        
    ax.annotate('-403 周烈王廿三年，三家分晉，戰國時代開始', 
                (-403, '衛'), 
                xytext=(-383, '衛'), 
                #xycoords='axes fraction', 
                xycoords='data', 
                arrowprops=dict(facecolor='red', shrink=.1), 
                fontsize=64)
    
    ax.annotate('-221 秦亡。春秋戰國結束。', 
                (-221, '吳'), 
                xytext=(-350, '吳'), 
                #xycoords='axes fraction', 
                xycoords='data', 
                arrowprops=dict(facecolor='red', shrink=.1), 
                fontsize=64)
    
    ax.annotate('戰國時代，', 
                (-403, '蔡'), 
                xytext=(-325, '蔡'), 
                #xycoords='axes fraction', 
                xycoords='data', 
                arrowprops=dict(facecolor='red', shrink=.1), 
                fontsize=64)

    ax.annotate('共182年', 
                (-221, '蔡'), 
                xytext=(-290, '蔡'), 
                #xycoords='axes fraction', 
                xycoords='data', 
                arrowprops=dict(facecolor='red', shrink=.1), 
                fontsize=64)

    print(ax.get_xticks())
    print(ax.get_yticks())

    ax.set_title('春秋戰國滅國年表')

In [ ]:
dict(facecolor='red', shrink=.1)

In [ ]:
def state_end_barh_1(end_df):
    #print(cser)
    fig, ax = plt.subplots(figsize=(45, 30))
    
    for c, (idx, row) in enumerate(end_df.iterrows()):
        p = ax.barh(row.state, row['end']+480, left=-480, color=tab_colors[c%len(tab_colors)])
        ax.bar_label(p, labels=[f"{row['lord']}"], label_type='center')
        ax.bar_label(p, labels=[f"{row['end']} {row['conqueror']}"], label_type='edge', padding=6)
        
    ax.set_xlabel('公元年')
    ax.set_ylabel('被滅國名')
    
    y2 = ax.secondary_yaxis('right')
    y2labels = [c[0] for c in end_df['conqueror'].str.extract(r'.*(.{1})$').values]
    y2.set_yticks(ax.get_yticks(), labels=y2labels)
    y2.set_ylabel('滅國名')

    x2 = ax.secondary_xaxis('top')
    x2.set_xticks([-403, -221], labels=['-403', '-221'])
    x2.set_xlabel('')

    ax.vlines([-403, -221], [0, 0], [18, 18], colors='black', linestyles='solid', label='', data=None)

    ax.annotate('',
                (-403, '衛'), 
                xytext=(-390, '衛'),
                xycoords='data', 
                arrowprops=dict(facecolor='red', arrowstyle='->'), 
                fontsize=64)
    ax.annotate('-403 周烈王廿三年，三家分晉，戰國時代開始', xy=(-312, '衛'), ha='center', va='center', fontsize=64)
    
    ax.annotate('',
                (-221, '陳'), 
                xytext=(-260, '陳'),
                xycoords='data', 
                arrowprops=dict(facecolor='red', arrowstyle='->'), 
                fontsize=64)
    ax.annotate('-221 六國亡。春秋戰國結束。', xy=(-312, '陳'), ha='center', va='center', fontsize=64)
    
    ax.annotate('',
                (-403, '蔡'), 
                xytext=(-221, '蔡'),
                xycoords='data', 
                arrowprops=dict(facecolor='red', arrowstyle='<->'), 
                fontsize=64)
    ax.annotate('戰國時代，共182年', xy=(-312, 14.3), ha='center', va='center', fontsize=64)
    
    ax.set_title('春秋戰國滅國年表')

In [ ]:
# state_end_barh_1(end_df_1_by_ending)

In [ ]:
def state_end_barh(end_df):
    #print(cser)
    fig, ax = plt.subplots(figsize=(15, 15))
    
    for idx, row in end_df.iterrows():
        p = ax.barh(row.state, row['end']+500, left=-500, color='skyblue')
        ax.bar_label(p, labels=[f"{row['end']} {row['conqueror']}"], label_type='edge')

    ax.set_title('春秋戰國滅國年表')

In [ ]:
state_order = end_df.sort_values(by='conqueror')['state']

In [ ]:
#ser1 = cser.loc[state_order]
#state_end_scatter(cser[-1::-1])
state_end_barh(end_df.loc[state_order.index])

In [ ]:
# Another way to get end_df fron df

# Group df by state
df_gb_state = df.groupby(by='state')

# Get last row of each group
end_df_data_1 = []

for k, v in df_gb_state:
    end_df_data_1.append(v.iloc[-1])

end_df_1 = pd.DataFrame(end_df_data_1)
    
end_df_1.sort_index()

In [ ]:
end_df.sort_index()

In [ ]:
end_df_1_by_ending = end_df_1.loc[end_df_1.sort_values(by='end', ascending=False).index]

In [ ]:
state_end_barh(end_df_1_by_ending)

In [ ]:
state_end_barh_1(end_df_1_by_ending)

In [ ]:
fig, ax = plt.subplots(layout='constrained', figsize=(8, 8))
dir(ax.xaxis)

In [ ]:
# data from https://allisonhorst.github.io/palmerpenguins/

import matplotlib.pyplot as plt
import numpy as np

species = ("Adelie", "Chinstrap", "Gentoo")
penguin_means = {
    'Bill Depth': (18.35, 18.43, 14.98),
    'Bill Length': (38.79, 48.83, 47.50),
    'Flipper Length': (189.95, 195.82, 217.19),
}

x = np.arange(len(species))  # the label locations
print(x)
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained', figsize=(8, 8))

for attribute, measurement in penguin_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

print(x + width)
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Length (mm)')
ax.set_title('Penguin attributes by species')
ax.set_xticks(x + width, species)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 250)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

# Create a simple plot
fig, ax = plt.subplots()
ax.plot([1, 2, 3, 4], [2, 4, 1, 3])

# Use a MultipleLocator to place ticks at multiples of 2 on the x-axis
#ax.xaxis.set_major_locator(MultipleLocator(2))
ax.set_xticks([1, 2, 3, 4])

# Show the plot
plt.show()


In [ ]:
# Create a simple plot
fig, ax = plt.subplots()
ax.plot([1, 2, 3, 4], [2, 4, 1, 3])

# Use a MultipleLocator to place ticks at multiples of 2 on the x-axis
ax.xaxis.set_major_locator(MultipleLocator(2))
#ax.set_xticks([1, 2, 3, 4])

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Horizontal bar plot with gaps
fig, ax = plt.subplots()
ax.broken_barh([(110, 30), (150, 10)], (10, 9), facecolors='tab:blue')
ax.broken_barh([(10, 50), (100, 20), (130, 10)], (20, 9),
               facecolors=('tab:orange', 'tab:green', 'tab:red'))
ax.set_ylim(5, 35)
ax.set_xlim(0, 200)
ax.set_xlabel('seconds since start')
ax.set_yticks([15, 25], labels=['Bill', 'Jim'])     # Modify y-axis tick labels
ax.grid(True)                                       # Make grid lines visible
ax.annotate('race interrupted', (61, 25),
            xytext=(0.8, 0.9), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', shrink=0.05),
            fontsize=16,
            horizontalalignment='right', verticalalignment='top')

plt.show()

# Your given pandas series
data = [-479, -478, -473, -472, -286, -256, -256, -230, -225, -223, -222, -222, -221, -207]
index = range(len(data))
series = pd.Series(data, index=index)

# Define the values you want to skip on the x-axis
#values_to_skip = [-286, -222]
values_to_skip = [-450, -400, -350, -300]

# Create a list of tick positions and labels, skipping the specified values
custom_ticks = [(value, str(value)) for value in series.values if value not in values_to_skip]

# Plotting with custom ticks
plt.barh(series.index, series.values+480, left=-480)

# Set custom ticks on the x-axis
plt.xticks(*zip(*custom_ticks))

# Display the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Your given pandas series
data = [-479, -478, -473, -472, -286, -256, -256, -230, -225, -223, -222, -222, -221, -207]
index = range(len(data))
series = pd.Series(data, index=index)

# Define the values you want to skip on the x-axis
values_to_skip = [-286, -222]

# Create a list of tick positions and labels, skipping the specified values
custom_ticks = [(value, str(value)) for value in series.values if value not in values_to_skip]

# Plotting with custom ticks
plt.barh(series.index, series.values)

# Set custom ticks on the x-axis
#plt.xticks(*zip(*custom_ticks))

# Display the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Your given pandas series
data = [-479, -478, -473, -472, -286, -256, -256, -230, -225, -223, -222, -222, -221, -207]
index = range(len(data))
series = pd.Series(data, index=index)

# Function to format tick labels, excluding values between 300 and 400
def custom_formatter(value, _):
    if 300 < value < 400:
        return ""
    return str(value)

# Plotting with custom x-axis formatter
fig, ax = plt.subplots()
ax.barh(series.index, series.values)
ax.xaxis.set_major_formatter(FuncFormatter(custom_formatter))

# Display the plot
plt.show()

In [ ]:
# importing matplotlib module and respective classes 
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator) 

x = [3, 2, 7, 4, 9] 
y = [10, 4, 7, 1, 2] 

fig, ax = plt.subplots() 

ax.set_title('Example Graph') 

ax.set_ylabel('y-AXIS') 
ax.set_xlabel('x-AXIS') 

ax.set_xlim(0, 10) 
ax.set_ylim(0, 10) 

# Make x-axis with major ticks that 
# are multiples of 11 and Label major 
# ticks with '% 1.2f' formatting 
ax.xaxis.set_major_locator(MultipleLocator(5)) 
ax.xaxis.set_major_formatter(FormatStrFormatter('% 1.1f')) 

# make x-axis with minor ticks that 
# are multiples of 1 and label minor 
# ticks with '% 1.2f' formatting 
ax.xaxis.set_minor_locator(MultipleLocator(1)) 
ax.xaxis.set_minor_formatter(FormatStrFormatter('% 1.0f')) 

ax.plot(x, y) 
plt.show() 